# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru/model/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error

from pipeline_config import *
from reversed_hgru_data_preprocess.preprocess_config import *

#from Prices.prices_over_time import *

In [3]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Results Data

In [4]:
with open(test_predictions_path, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(HRNNpath, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [5]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_7248/483307922.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_7248/483307922.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_7248/483307922.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [6]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [7]:
len(all_data_test_dict.keys())

357

In [8]:
categories_per_indent_dict ={key: categories_per_indent_dict[key] for key in categories_per_indent_dict.keys() if key in [0,1,2,3]}
cat_ids = list(categories_per_indent_dict.values())

flat_cat_id = [item for sublist in cat_ids for item in sublist]

all_categories = []
for cat_id in flat_cat_id:
    all_categories.append(category_id_to_name_dict[cat_id])

#all_categories = list(all_data_test_dict.keys())

In [9]:
all_categories = list(all_data_test_dict.keys())

In [10]:
len(all_categories)

357

In [11]:
def plot_results(all_data_dict, categories):
    category_samples = random.sample(categories, 20)+['All items']
    #category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')
        print(f'Corr is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [12]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    total_mse = sum(mse_lst)
    avg_mse = total_mse/len(list(all_data_test_dict.keys()))
    avg_rmse = np.sqrt(avg_mse)
    
    return avg_rmse

In [13]:
avg_rmse(all_data_test_dict)

2.14718571317589

In [14]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [15]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 19


46.352644697783454

In [16]:
len(all_categories)

357

In [17]:
plot_results(all_data_test_dict, all_categories)

Category is: Electricity
RMSE is: 1.0847428106010906


Category is: Sewing machines, fabric and supplies
RMSE is: 4.849161515498004


Category is: Other fats and oils including peanut butter
RMSE is: 2.172248452233387


Category is: Housing at school, excluding board
RMSE is: 0.16352134022508713


Category is: All items
RMSE is: 0.4094045801870134


...

Category is: Roasted coffee
RMSE is: 1.6213036642213874


Category is: Other appliances
RMSE is: 1.6541297303830218


Category is: Other linens
RMSE is: 2.327127559951154


Category is: Frozen noncarbonated juices and drinks
RMSE is: 1.5767036336007036


Category is: All items
RMSE is: 0.4094045801870134


-----------

# Horizon Evaluation: